In [1]:
import pandas as pd
import numpy as np



import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
plt.rcParams['font.family'] = 'NanumGothic'




import folium
import json
from folium.plugins import MarkerCluster
import geopandas as gpd
from shapely.geometry import Point, Polygon,MultiPolygon
from pyproj import Proj




import warnings
warnings.filterwarnings('ignore')

C:\Users\dblab\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dblab\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\dblab\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# ·

In [3]:
gdf = gpd.GeoDataFrame.from_file('../data/seoul_geo.geojson', encoding='utf-8')

#다른 환경에서 아래 코드를 돌릴 때 pyProj오류가 발생할 수 있으므로 아래 코드를 주석해제하고 실행
import os

os.environ['PROJ_LIB']=r"C:/Users/dblab/Anaconda3/Library/share"  #C:/ .... / Anaconda3 사이의 주소는 본인 PC에 맞게 설정

gdf.crs = {'init':'epsg:4326'}
gdf = gdf.to_crs({'init':'epsg:5179'})

# Point(126.989574,37.569006).within(gdf.geometry[0].buffer(0))
gdf['dong'] = gdf['adm_nm'].str.split().str[2]
def return_office_dong(point):
    dong_df = pd.DataFrame()
    dong_df['dong'] = gdf['adm_nm'].str.split().str[2]
    dong_list = dong_df.dong.tolist()
    dong_list
    for i,dong in enumerate(dong_list):
        if(point.within(gdf.geometry[i].buffer(0)) == True):
            return str(dong_list[i])
    else:
        return None

In [4]:
전체행정동list = gdf['dong'].unique()
전체행정동list

array(['사직동', '삼청동', '부암동', '평창동', '무악동', '교남동', '가회동', '종로1·2·3·4가동',
       '종로5·6가동', '이화동', '창신1동', '창신2동', '창신3동', '숭인1동', '숭인2동', '청운효자동',
       '혜화동', '소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동', '신당5동',
       '황학동', '중림동', '신당동', '다산동', '약수동', '청구동', '동화동', '후암동', '용산2가동',
       '남영동', '원효로2동', '효창동', '용문동', '이촌1동', '이촌2동', '이태원1동', '이태원2동',
       '서빙고동', '보광동', '청파동', '원효로1동', '한강로동', '한남동', '왕십리2동', '마장동',
       '사근동', '행당1동', '행당2동', '응봉동', '금호1가동', '금호4가동', '성수1가1동', '성수1가2동',
       '성수2가1동', '성수2가3동', '송정동', '용답동', '왕십리도선동', '금호2·3가동', '옥수동',
       '화양동', '군자동', '중곡1동', '중곡2동', '중곡3동', '중곡4동', '능동', '구의1동', '구의2동',
       '구의3동', '광장동', '자양1동', '자양2동', '자양3동', '자양4동', '회기동', '휘경1동',
       '휘경2동', '청량리동', '용신동', '제기동', '전농1동', '전농2동', '답십리2동', '장안1동',
       '장안2동', '이문1동', '이문2동', '답십리1동', '면목2동', '면목4동', '면목5동', '면목7동',
       '상봉1동', '상봉2동', '중화1동', '중화2동', '묵1동', '묵2동', '망우3동', '신내1동',
       '신내2동', '면목본동', '면목3·8동', '망우본동', '돈암1동', '돈암2동', '안암동', '보문동',
   

# 통계데이터들 전처리

## 동별 분기별 외국인 수 데이터 전처리

In [5]:
외국인데이터 = pd.read_csv('../통계데이터/서울 외국인 데이터(20년도1분기~20년도4분기).csv')
외국인데이터 = 외국인데이터[(외국인데이터['동'] != '합계') & (외국인데이터['동'] != '소계')].reset_index(drop=True)
print(set(외국인데이터['동'].unique()) - set(전체행정동list))
외국인데이터['년도'] = 외국인데이터['시점'].str.split('·').str[0].astype('int')
외국인데이터['분기'] = 외국인데이터['시점'].apply(lambda x: int(x[6:7]))
외국인데이터 = 외국인데이터.drop(['구분','연령','시점'],axis=1)
외국인데이터.columns = ['자치구','행정동','총 외국인 수','년도','분기']
외국인데이터.to_csv('../통계데이터_전처리후/서울 외국인 데이터(20년도1분기~20년도4분기).csv',encoding='utf-8-sig')

set()


## 서울 대학교 데이터

In [6]:
서울대학교데이터 = pd.read_csv('../point_Polygon_전처리후_데이터/서울시 대학 및 전문대학 DB 정보 (한국어).csv')
print(set(서울대학교데이터['행정동'].unique()) - set(전체행정동list))
서울대학교데이터.rename(columns = {'행정구' : '자치구'}, inplace = True)
동별대학교갯수 = 서울대학교데이터.pivot_table(values='키값',aggfunc='count',index=['자치구','행정동'])
동별대학교갯수.to_csv('../통계데이터_전처리후/서울시 대학 및 전문대학 DB 정보 (한국어).csv',encoding='utf-8-sig')

set()


## 금융기관 수

In [7]:
금융기관수 = pd.read_excel('../통계데이터/서울시 동별 금융기관 수.xls')
금융기관수 = 금융기관수[['기간','자치구','동','금융기관']]
금융기관수 = 금융기관수[(금융기관수['동']!='동') & (금융기관수['동']!='합계') & (금융기관수['동']!='소계')].reset_index(drop=True)
print(set(금융기관수['동'].unique()) - set(전체행정동list))
금융기관수.columns = [['년도','자치구','행정동','금융기관 수']]
금융기관수.to_csv('../통계데이터_전처리후/서울시 동별 금융기관 수.csv',encoding='utf-8-sig')

set()


## 동별 기초생활 수급자

In [8]:
기초생활수급자 = pd.read_csv('../통계데이터/서울시 동별 기초생활 수급자.csv')
기초생활수급자 = 기초생활수급자[(기초생활수급자['동']!='동') & (기초생활수급자['동']!='본청') & 
                 (기초생활수급자['동']!='합계') & (기초생활수급자['동']!='소계') & (기초생활수급자['동']!='기타')].reset_index(drop=True)
print(set(기초생활수급자['동'].unique()) - set(전체행정동list))
기초생활수급자 = 기초생활수급자[['기간','자치구','동','총 수급자']]
기초생활수급자.columns = [['년도','자치구','행정동','총 수급자 수']]
기초생활수급자.to_csv('../통계데이터_전처리후/서울시 동별 기초생활 수급자.csv',encoding='utf-8-sig')

set()


## 동별 노령화

In [9]:
동별노령화 = pd.read_csv('../통계데이터/서울시 동별 노령화 데이터.csv')
동별노령화 = 동별노령화[(동별노령화['동'] != '소계') & (동별노령화['동'] != '합계')].reset_index(drop=True)
print(set(동별노령화['동'].unique()) - set(전체행정동list))
동별노령화.rename(columns = {'기간' : '년도', '동':'행정동'}, inplace = True)
동별노령화.to_csv('../통계데이터_전처리후/서울시 동별 노령화 데이터.csv',encoding='utf-8-sig')

set()


## 서울시 인구 밀도 데이터

In [10]:
인구밀도 = pd.read_excel('../통계데이터/서울시 인구밀도(동별)통계.xls')
인구밀도 = 인구밀도[(인구밀도['동'] != '소계') & (인구밀도['동'] != '합계')].reset_index(drop=True)
print(set(인구밀도['동'].unique()) - set(전체행정동list))
인구밀도.columns = [['년도','자치구','행정동','인구 수','면적','인구밀도']]
인구밀도.to_csv('../통계데이터_전처리후/서울시 인구밀도(동별)통계.csv',encoding='utf-8-sig')

set()


## 서울시 동별 평균 연령

In [11]:
동별평균연령 = pd.read_excel('../통계데이터/서울 동별 평균연령.xls')
동별평균연령 = 동별평균연령[(동별평균연령['동'] != '소계') & (동별평균연령['동'] != '합계')].reset_index(drop=True)
print(set(동별평균연령['동'].unique()) - set(전체행정동list))
동별평균연령['년도'] = 동별평균연령['기간'].apply(lambda x: int(x[:4]))
동별평균연령['분기'] = 동별평균연령['기간'].apply(lambda x: int(x[5:6]))
동별평균연령.columns = [['기간','자치구','행정동','전체평균연령','남자_전체평균연령','여자_전체평균연령','년도','분기']]
동별평균연령.to_csv('../통계데이터_전처리후/서울 동별 평균연령.csv',encoding='utf-8-sig')

set()


## 동별 주차장

In [12]:
동별주차장 =  pd.read_excel('../통계데이터/동별주차장.xls')
동별주차장 = 동별주차장[(동별주차장['동'] != '소계') & 
                (동별주차장['동'] != '합계') & (동별주차장['동'] != '동')].reset_index(drop=True)
# print(set(동별주차장['동'].unique()) - set(전체행정동list))
동별주차장 = 동별주차장[['기간','자치구','동','합계','합계.1']]
동별주차장.columns = [['년도','자치구','행정동','주차장_개소','주차장_면수']]
동별주차장.to_csv('../통계데이터_전처리후/동별주차장.csv',encoding='utf-8-sig')

## 동별 주택 종류별 통계

In [13]:
주택종류별통계 = pd.read_excel('../통계데이터/주택종류별(동별)통계.xls')
주택종류별통계 = 주택종류별통계[['기간','자치구','동','종류별 주택수']]
주택종류별통계 = 주택종류별통계[(주택종류별통계['동'] != '소계') & 
                (주택종류별통계['동'] != '합계') & (주택종류별통계['동'] != '동')].reset_index(drop=True)
주택종류별통계.columns = [['년도','자치구','행정동','전체종류주택합계']]
주택종류별통계.to_csv('../통계데이터_전처리후/주택종류별(동별)통계.csv',encoding='utf-8-sig')

## 세대원수별 세대수 통계 데이터

In [14]:
세대수 = pd.read_excel('../통계데이터/서울 세대원수별 세대수.xls')
세대수 = 세대수[(세대수['동']!='계') & (세대수['동']!='합계')].reset_index(drop=True)
print(set(세대수['동'].unique()) - set(전체행정동list))
세대수['년도'] = 세대수['기간'].apply(lambda x: int(x[:4]))
세대수['분기'] = 세대수['기간'].apply(lambda x: int(x[5:6]))
세대수.rename(columns = { '동':'행정동'}, inplace = True)
세대수 = 세대수[['년도','분기','자치구','행정동','전체세대수','1인세대','2인세대','3인세대','4인세대','5인세대','6인세대']]
세대수.to_csv('../통계데이터_전처리후/서울 세대원수별 세대수.csv',encoding='utf-8-sig')

set()


## 동별평균연령

In [15]:
평균연령 = pd.read_excel('../통계데이터/동별평균연령.xls')
평균연령 = 평균연령[(평균연령['동']!='소계') & (평균연령['동']!='합계')].reset_index(drop=True)
print(set(평균연령['동'].unique()) - set(전체행정동list))
평균연령['년도'] = 평균연령['기간'].apply(lambda x: int(x[:4]))
평균연령['분기'] = 평균연령['기간'].apply(lambda x: int(x[5:6]))
평균연령.rename(columns = { '동':'행정동','남자':'남자_평균연령','여자':'여자_평균연령'}, inplace = True)
평균연령 = 평균연령.drop('기간',axis=1)
평균연령.to_csv('../통계데이터_전처리후/동별평균연령.csv',encoding='utf-8-sig')

set()


## 세대원수별 세대수 통계 데이터

In [16]:
세대수 = pd.read_excel('../통계데이터/서울 세대원수별 세대수.xls')
세대수 = 세대수[(세대수['동']!='계') & (세대수['동']!='합계')].reset_index(drop=True)
print(set(세대수['동'].unique()) - set(전체행정동list))
세대수['년도'] = 세대수['기간'].apply(lambda x: int(x[:4]))
세대수['분기'] = 세대수['기간'].apply(lambda x: int(x[5:6]))
세대수.rename(columns = { '동':'행정동'}, inplace = True)
세대수 = 세대수[['년도','분기','자치구','행정동','전체세대수','1인세대','2인세대','3인세대','4인세대','5인세대','6인세대']]
세대수.to_csv('../통계데이터_전처리후/서울 세대원수별 세대수.csv',encoding='utf-8-sig')

set()


## 구별 학급당 학생 수 

In [17]:
구별학급당학생수 = pd.read_excel('../data/구별 학급당 학생 수.xls')
구별학급당학생수 = 구별학급당학생수.drop(['유치원.2','초등학교.2','중학교.2','고등학교.2'],axis=1)
구별학급당학생수.columns = ['년도','행정구','유치원생_수','유치원학급수','초등학생수','초등학교학급수','중학생수','중학교학급수','고등학생수','고등학교학급수']
구별학급당학생수 = 구별학급당학생수.drop([0,1]).reset_index(drop=True)
구별학급당학생수.to_csv('../통계데이터_전처리후/구별학급당학생수.csv',encoding='utf-8-sig')

## 구별 의료기관

In [18]:
구별의료기관 = pd.read_excel('../data/구별_의료기관.xls')
구별의료기관 = 구별의료기관[['기간','자치구','계','계.1','종합병원','종합병원.1','병원','병원.1','의원','의원.1']]
구별의료기관.columns = ['년도','행정구','총병원수','총병상수','총종합병원수','총종합병원병상수','총일반병원수','총일반병원병상수','총의원수','총의원병상수']
구별의료기관 = 구별의료기관.drop(0).reset_index(drop=True)
구별의료기관.to_csv('../통계데이터_전처리후/구별의료기관.csv',encoding='utf-8-sig')

# PPI 전처리를 거친 데이터들 통계시키기

## 동별지하철역 수

In [19]:
서울시지하철역 = pd.read_csv('../point_Polygon_전처리후_데이터/서울시지하철역.csv').drop('Unnamed: 0',axis=1)


In [20]:
동별지하철역갯수 = 서울시지하철역.pivot_table(values='X좌표',aggfunc='count',index='행정동')
동별지하철역갯수.columns = ['동별 지하철역 수']
동별지하철역갯수.to_csv('../통계데이터_전처리후/동별지하철역수.csv',encoding='utf-8-sig')